<a href="https://colab.research.google.com/github/Tarun1220/My-HDS5210_InClass/blob/master/Copy_of_midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [20]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [21]:
def calculate_bmi(weight_kg, height_m):
    """
    Calculate BMI (Body Mass Index) using weight in kilograms and height in meters.

    : weight_kg: Weight in kilograms
    : height_m: Height in meters
    :return: Calculated BMI value (float)
    :raises ValueError: If weight or height are non-positive
    """
    if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height must be positive numbers.")

    return weight_kg / (height_m ** 2)


In [22]:
# Test Case 1:
print(calculate_bmi(70, 1.75))

# Test Case 2:
print(calculate_bmi(50, 1.6))

# Test Case 3:
try:
    print(calculate_bmi(70, -1.75))
except ValueError as e:
    print(e)

# Test Case 4:
try:
    print(calculate_bmi(0, 1.75))
except ValueError as e:
    print(e)


22.857142857142858
19.531249999999996
Weight and height must be positive numbers.
Weight and height must be positive numbers.


### Step 2: Calculate BODE Score

In [23]:
def bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters):
    """
    Calculate BODE score for a patient based on BMI, FEV1%, dyspnea, and 6-minute walk distance.

    : bmi: Body Mass Index (float)
    : fev_pct: FEV1 percentage (float)
    : dyspnea_description: A description of dyspnea (breathlessness level)
    : distance_in_meters: Distance walked in meters during a 6-minute walk test
    :return: BODE score (int)
    :raises ValueError: If input data is invalid
    """

    # BMI score
    if bmi > 21:
        bmi_score = 0
    else:
        bmi_score = 1

    # FEV1 score
    if fev_pct >= 65:
        fev_score = 0
    elif fev_pct >= 50:
        fev_score = 1
    elif fev_pct >= 36:
        fev_score = 2
    elif fev_pct < 36:
        fev_score = 3
    else:
        raise ValueError("Invalid FEV1 percentage.")

    # Dyspnea score based on description
    dyspnea_mapping = {
        "Only breathless with strenuous exercise": 0,
        "Breathless when hurrying or walking uphill": 1,
        "Walks slower, stops for breath": 2,
        "Stops for breath after 100 yards or a few minutes on level ground": 3,
        "Too breathless to leave house or while dressing": 4
    }

    if dyspnea_description not in dyspnea_mapping:
        raise ValueError("Invalid dyspnea description provided.")

    dyspnea_score = dyspnea_mapping[dyspnea_description]

    # 6-minute walk distance score
    if distance_in_meters >= 350:
        distance_score = 0
    elif distance_in_meters >= 250:
        distance_score = 1
    elif distance_in_meters >= 150:
        distance_score = 2
    elif distance_in_meters < 150:
        distance_score = 3
    else:
        raise ValueError("Invalid 6-minute walk distance.")

    # Total BODE score
    return bmi_score + fev_score + dyspnea_score + distance_score


In [24]:
# Test Case 1:
bmi = calculate_bmi(70, 1.75)
fev_pct = 65
dyspnea_description = "Breathless when hurrying or walking uphill"
distance_in_meters = 300
print(bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters))

# Test Case 2:
bmi = calculate_bmi(55, 1.60)
fev_pct = 35
dyspnea_description = "Stops for breath after 100 yards or a few minutes on level ground"
distance_in_meters = 100
print(bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters))

# Test Case 3:
try:
    print(bode_score(22, 60, "Unknown description", 200))
except ValueError as e:
    print(e)

# Test Case 4:
try:
    print(bode_score(22, -10, "Breathless when hurrying or walking uphill", 300))
except ValueError as e:
    print(e)


2
9
Invalid dyspnea description provided.
5


### Step 3: Calculate BODE Risk

In [25]:
def bode_risk(score):
    """
    Map a BODE score to a survival risk percentage based on predefined ranges.

    : score: BODE score (int)
    :return: 4-year survival risk percentage (int)
    :raises ValueError: If the score is outside the valid range (0–10)
    """
    if not (0 <= score <= 10):
        raise ValueError("BODE score must be between 0 and 10.")

    if 0 <= score <= 2:
        return 80
    elif 3 <= score <= 4:
        return 67
    elif 5 <= score <= 6:
        return 57
    elif 7 <= score <= 10:
        return 18


In [26]:
# Test Case 1:
print(bode_risk(2))

# Test Case 2:
print(bode_risk(3))

# Test Case 3:
print(bode_risk(5))
# Test Case 4:
print(bode_risk(8))
# Test Case 5:
try:
    print(bode_risk(11))
except ValueError as e:
    print(e)

# Test Case 6:
try:
    print(bode_risk(-1))
except ValueError as e:
    print(e)


80
67
57
18
BODE score must be between 0 and 10.
BODE score must be between 0 and 10.


### Step 4: Load Hospital Data

In [27]:
import csv
import json

# patients data
def load_patient_data(file_path):
    """
    Load patient data from a CSV file
    """
    required_columns = {'NAME', 'HEIGHT_M', 'WEIGHT_KG', 'fev_pct', 'dyspnea_description', 'distance_in_meters', 'hospital'}
    patient_data = []

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)

        # Check for missing columns
        missing_cols = required_columns - set(reader.fieldnames)
        if missing_cols:
            raise ValueError(f"Missing columns: {missing_cols}")

        for row in reader:
            try:
                patient_data.append({
                    'NAME': row['NAME'],
                    'HEIGHT_M': float(row['HEIGHT_M']),
                    'WEIGHT_KG': float(row['WEIGHT_KG']),
                    'fev_pct': float(row['fev_pct']),
                    'dyspnea_description': row['dyspnea_description'],
                    'distance_in_meters': float(row['distance_in_meters']),
                    'hospital': row['hospital']
                })
            except (ValueError, KeyError) as e:
                name = row.get('NAME', 'Unknown')
                print(f"Skipping {name} due to error: {e}")

    return patient_data

# Load patient data
try:
    patients = load_patient_data('/content/patient.csv')
    print(f"Loaded {len(patients)} patients from '/content/patient.csv'.")
except ValueError as e:
    print(f"Error loading patient data: {e}")

# hospital data
def load_hospital_info(json_path):
    """
    Load hospital information from a JSON file.
    """
    try:
        with open(json_path, 'r') as json_file:
            return json.load(json_file)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")

# Load hospital data
try:
    hospitals = load_hospital_info('/content/hospitals (2).json')
    print(f"Loaded {len(hospitals)} hospitals from '/content/hospitals (2).json'.")
except ValueError as e:
    print(f"Error loading hospital data: {e}")


Loaded 1000 patients from '/content/patient.csv'.
Loaded 3 hospitals from '/content/hospitals (2).json'.


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [28]:
import csv
import json

# Step 1: Define file paths
patient_csv = "/content/patient.csv"
hospital_json = "/content/hospitals (2).json"

# Step 2: Define a function to calculate the BODE Score and Risk
def calculate_bode_score(patient):

    bode_score = (patient['HEIGHT_M'] + patient['WEIGHT_KG']) / 2
    bode_risk = bode_score / 10
    return bode_score, bode_risk

# Step 3: Load the patient data from CSV
def load_patient_data(filename):
    patient_data = []
    with open(filename, mode='r') as file:
        csv_reader = csv.DictReader(file)

        # Print the headers (column names)
        headers = csv_reader.fieldnames
        print("CSV Headers:", headers)

        for row in csv_reader:
            row['HEIGHT_M'] = float(row['HEIGHT_M'])
            row['WEIGHT_KG'] = float(row['WEIGHT_KG'])
            patient_data.append(row)

    return patient_data

# Step 4: Load the hospital data from JSON
def load_hospital_data(filename):
    with open(filename, mode='r') as file:
        hospital_data = json.load(file)
    return hospital_data

# Step 5: Process the patients and calculate BODE Scores and Risks
def process_patients_and_hospitals(patient_data, hospital_data):
    patient_results = []
    hospital_aggregates = {}

    # Step 6: Process each patient
    for patient in patient_data:
        # Calculate the BODE score and risk for each patient
        bode_score, bode_risk = calculate_bode_score(patient)
        patient_id = patient['NAME']
        hospital_id = patient['hospital']

        # Store patient result
        patient_results.append([patient_id, hospital_id, bode_score, bode_risk])

        # Aggregate data by hospital
        if hospital_id not in hospital_aggregates:
            hospital_aggregates[hospital_id] = {
                'total_bode_score': 0,
                'total_bode_risk': 0,
                'num_patients': 0
            }

        hospital_aggregates[hospital_id]['total_bode_score'] += bode_score
        hospital_aggregates[hospital_id]['total_bode_risk'] += bode_risk
        hospital_aggregates[hospital_id]['num_patients'] += 1

    # Step 7: Calculate the averages for each hospital
    hospital_output_list = []
    for hospital_id, aggregates in hospital_aggregates.items():
        avg_bode_score = aggregates['total_bode_score'] / aggregates['num_patients']
        avg_bode_risk = aggregates['total_bode_risk'] / aggregates['num_patients']
        hospital_output_list.append([hospital_id, avg_bode_score, avg_bode_risk, aggregates['num_patients']])

    return patient_results, hospital_output_list

# Step 8: Write the results to CSV files
def write_csv(filename, data, headers=None):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if headers:
            writer.writerow(headers)
        writer.writerows(data)

# Step 9: Load data, process it, and save the results
patient_data = load_patient_data(patient_csv)
hospital_data = load_hospital_data(hospital_json)

# Process the data and get the results
patient_results, hospital_output_list = process_patients_and_hospitals(patient_data, hospital_data)

# Write the patient and hospital results to their respective CSV files
write_csv("patient_output.csv", patient_results, headers=["PATIENT_NAME", "HOSPITAL", "BODE_SCORE", "BODE_RISK"])
write_csv("hospital_output.csv", hospital_output_list, headers=["HOSPITAL", "AVG_BODE_SCORE", "AVG_BODE_RISK", "NUM_PATIENTS"])

# Output for verification (first few lines)
print("Patient Results:")
for row in patient_results[:5]:
    print(row)

print("\n Hospital Results:")
for row in hospital_output_list[:5]:
    print(row)


CSV Headers: ['NAME', 'SSN', 'LANGUAGE', 'JOB', 'HEIGHT_M', 'WEIGHT_KG', 'fev_pct', 'dyspnea_description', 'distance_in_meters', 'hospital']
Patient Results:
['Vanessa Roberts', "ST.LUKE'S", 46.0, 4.6]
['Christopher Fox', 'SAINT LOUIS UNIVERSITY', 42.365, 4.2365]
['Benjamin Johnston', 'BJC', 48.26, 4.826]
['Christopher Hernandez', 'MISSOURI BAPTIST', 41.61, 4.161]
['Valerie Burch', 'BJC WEST COUNTY', 43.144999999999996, 4.3145]

 Hospital Results:
["ST.LUKE'S", 49.28707317073169, 4.92870731707317, 164]
['SAINT LOUIS UNIVERSITY', 49.36060975609756, 4.936060975609758, 164]
['BJC', 49.58717391304347, 4.95871739130435, 184]
['MISSOURI BAPTIST', 49.856801242236, 4.985680124223601, 161]
['BJC WEST COUNTY', 49.26999999999999, 4.9270000000000005, 171]
